In [1]:
import os
import sys
sys.path.append('../')

import numpy as np
from matplotlib.dates import DateFormatter
import matplotlib.pyplot as plt
import pandas as pd
from pprint import pprint
import datetime
import matplotlib

import src.io as sio
import src.preprocessing as spp
import src.fitting as sft
import ipympl

In [2]:
FIRST_COOLDOWN_FOLDER = sio.get_folderpath("20201217_FirstCooldown")

In [24]:
# Temperature
df1 = sio.read_tm224_data("temperature-monitoring01_LN2.xls", FIRST_COOLDOWN_FOLDER)
df2 = sio.read_tm224_data("temperature-monitoring02_LN2.xls", FIRST_COOLDOWN_FOLDER)
dft = pd.concat([df1])

df1 = sio.read_tpg_data("pressure-monitoring01_LN2", FIRST_COOLDOWN_FOLDER)
dfp = pd.concat([df1])

In [28]:
fig, ax = plt.subplots()

myFmt = DateFormatter("%H:%M\n%a %d")
ax.xaxis.set_major_formatter(myFmt)

ax.set_ylabel(r"Temperature (K)")
ax.plot(dft["MPL_datetimes"], dft["Baseplate"], color="C1", label="Baseplate")
# Sensor location was changed
ax.plot(dft["MPL_datetimes"], dft["Input C1"], color="C1")
ax.plot(dft["MPL_datetimes"], dft["Magnet Base"], color="C3", label="Magnet Base")
ax.plot(dft["MPL_datetimes"], dft["Tip Holder"], color="C2", label="Tip Holder")

ax.legend()
ax.set_ylim([290, 310])

ax2 = ax.twinx()
myFmt = DateFormatter("%H:%M\n%a-%d")
ax2.xaxis.set_major_formatter(myFmt)

# Pressure
ax2.set_ylabel(r"Main Pressure (mbar)", color="C0")
ax2.set_yscale("log", base=10)
ax2.tick_params(axis='y', labelcolor="C0")
ax2.plot(dfp["MPL_datetimes"], dfp["Main"], "-", color="C0")

sio.savefig("cooldown_LN2_outer_bath", FIRST_COOLDOWN_FOLDER)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
fig, ax = plt.subplots(figsize=(10, 8))

myFmt = DateFormatter("%H:%M\n%a-%d")
ax.xaxis.set_major_formatter(myFmt)

# Pressure
ax.set_ylabel(r"Main Pressure (mbar)", color="C0")
ax.set_yscale("log", base=10)
ax.tick_params(axis='y', labelcolor="C0")
ax.plot(dfp["MPL_datetimes"], dfp["Main"], "-", color="C0")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Cooling from 300 K to 80 K (LN2)

In [28]:
# Temperature
df1 = sio.read_tm224_data("temperature-monitoring01_LN2.xls", FIRST_COOLDOWN_FOLDER)
df2 = sio.read_tm224_data("temperature-monitoring02_LN2.xls", FIRST_COOLDOWN_FOLDER)
df3 = sio.read_tm224_data("temperature-monitoring03_LN2.xls", FIRST_COOLDOWN_FOLDER)
df4 = sio.read_tm224_data("temperature-monitoring04_LN2.xls", FIRST_COOLDOWN_FOLDER)
df5 = sio.read_tm224_data("temperature-monitoring05_LN2.xls", FIRST_COOLDOWN_FOLDER)
df6 = sio.read_tm224_data("temperature-monitoring06_LN2.xls", FIRST_COOLDOWN_FOLDER)
dft = pd.concat([df1, df2, df3, df4, df5, df6])

In [29]:
fig, ax = plt.subplots()

myFmt = DateFormatter("%H:%M\n%a %d")
ax.xaxis.set_major_formatter(myFmt)

ax.set_ylabel(r"Temperature (K)")
ax.plot(dft["MPL_datetimes"], dft["Baseplate"], color="C1", label="Baseplate")
# Sensor location was changed
ax.plot(dft["MPL_datetimes"], dft["Input C1"], color="C1")
ax.plot(dft["MPL_datetimes"], dft["Magnet Base"], color="C3", label="Magnet Base")
ax.plot(dft["MPL_datetimes"], dft["Tip Holder"], color="C2", label="Tip Holder")

ax.legend()

sio.savefig("cooldown_300K_to80K_LN2", FIRST_COOLDOWN_FOLDER)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Cooling from 80 K to 4 K (LHe)

In [31]:
# Temperature
df7 = sio.read_tm224_data("temperature-monitoring07_LN2.xls", FIRST_COOLDOWN_FOLDER)
df8 = sio.read_tm224_data("temperature-monitoring08_LN2.xls", FIRST_COOLDOWN_FOLDER)
df9 = sio.read_tm224_data("temperature-monitoring09_LN2.xls", FIRST_COOLDOWN_FOLDER)
df10 = sio.read_tm224_data("temperature-monitoring10_LN2.xls", FIRST_COOLDOWN_FOLDER)
df11 = sio.read_tm224_data("temperature-monitoring11_LN2.xls", FIRST_COOLDOWN_FOLDER)
df12 = sio.read_tm224_data("temperature-monitoring12_LN2.xls", FIRST_COOLDOWN_FOLDER)
df13 = sio.read_tm224_data("temperature-monitoring13_LN2.xls", FIRST_COOLDOWN_FOLDER)
df14 = sio.read_tm224_data("temperature-monitoring14_LN2.xls", FIRST_COOLDOWN_FOLDER)
df15 = sio.read_tm224_data("temperature-monitoring15_LN2.xls", FIRST_COOLDOWN_FOLDER)
df16 = sio.read_tm224_data("temperature-monitoring16_LHe.xls", FIRST_COOLDOWN_FOLDER)

dft = pd.concat([df7, df8, df9, df10, df11, df12, df13, df14, df15, df16])

In [32]:
fig, ax = plt.subplots()

myFmt = DateFormatter("%H:%M\n%a %d")
ax.xaxis.set_major_formatter(myFmt)

ax.set_ylabel(r"Temperature (K)")
ax.plot(dft["MPL_datetimes"], dft["Baseplate"], color="C1", label="Baseplate")
ax.plot(dft["MPL_datetimes"], dft["Magnet Base"], color="C3", label="Magnet Base")
ax.plot(dft["MPL_datetimes"], dft["Tip Holder"], color="C2", label="Tip Holder")

ax.legend()
sio.savefig("cooldown_80K_emptyLN2_LHe", FIRST_COOLDOWN_FOLDER)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Hold time at 4 K (LHe)

In [34]:
# Temperature
df17 = sio.read_tm224_data("temperature-monitoring17_LHe.xls", FIRST_COOLDOWN_FOLDER)
df18 = sio.read_tm224_data("temperature-monitoring18_LHe.xls", FIRST_COOLDOWN_FOLDER)

dft = pd.concat([df17, df18])

In [35]:
fig, ax = plt.subplots()

myFmt = DateFormatter("%H:%M\n%a %d")
ax.xaxis.set_major_formatter(myFmt)

ax.set_ylabel(r"Temperature (K)")
ax.plot(dft["MPL_datetimes"], dft["Baseplate"], color="C1", label="Baseplate")
ax.plot(dft["MPL_datetimes"], dft["Magnet Base"], color="C3", label="Magnet Base")
ax.plot(dft["MPL_datetimes"], dft["Tip Holder"], color="C2", label="Tip Holder")

ax.legend()
sio.savefig("holdtime_LHe", FIRST_COOLDOWN_FOLDER)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Base pressure

In [43]:
df1 = sio.read_tpg_data("pressure-monitoring04_LHe", FIRST_COOLDOWN_FOLDER)
df2 = sio.read_tpg_data("pressure-monitoring05_LHe", FIRST_COOLDOWN_FOLDER)
df3 = sio.read_tpg_data("pressure-monitoring06_LHe", FIRST_COOLDOWN_FOLDER)
dfp = pd.concat([df2, df3])

In [55]:
fig, ax = plt.subplots()

myFmt = DateFormatter("%H:%M\n%a %d")
ax.xaxis.set_major_formatter(myFmt)

# Pressure
ax.set_ylabel(r"Main Pressure (mbar)")
ax.set_yscale("log", base=10)
ax.plot(dfp["MPL_datetimes"], dfp["Main"], "-", color="C0")
ax.set_ylim([5e-10, 4e-9])
sio.savefig("pressure_LHe", FIRST_COOLDOWN_FOLDER)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Other stuff

In [ ]:
df1 = sio.read_tpg_data("pressure-monitoring01_LN2", FIRST_COOLDOWN_FOLDER)
df2 = sio.read_tpg_data("pressure-monitoring02_LN2", FIRST_COOLDOWN_FOLDER)
df3 = sio.read_tpg_data("pressure-monitoring03_LN2", FIRST_COOLDOWN_FOLDER)
dfp = pd.concat([df1, df2, df3])

# Temperature
df1 = sio.read_tm224_data("temperature-monitoring01_LN2.xls", FIRST_COOLDOWN_FOLDER)
df2 = sio.read_tm224_data("temperature-monitoring02_LN2.xls", FIRST_COOLDOWN_FOLDER)
df3 = sio.read_tm224_data("temperature-monitoring03_LN2.xls", FIRST_COOLDOWN_FOLDER)
df4 = sio.read_tm224_data("temperature-monitoring04_LN2.xls", FIRST_COOLDOWN_FOLDER)
df5 = sio.read_tm224_data("temperature-monitoring05_LN2.xls", FIRST_COOLDOWN_FOLDER)
df6 = sio.read_tm224_data("temperature-monitoring06_LN2.xls", FIRST_COOLDOWN_FOLDER)
dft = pd.concat([df1, df2, df3, df4, df5, df6])

## Alternative way using text matching
Note that this is slow and expensive

In [ ]:
dfp = pd.DataFrame()
for file in sio.get_filenames_matching("pressure-monitoring", FIRST_COOLDOWN_FOLDER):
    dfp = dfp.append(sio.read_tpg_data(file, FIRST_COOLDOWN_FOLDER), ignore_index=True)

In [ ]:
dft = pd.DataFrame()
for file in sio.get_filenames_matching("temperature-monitoring", FIRST_COOLDOWN_FOLDER):
    dft = dft.append(sio.read_tm224_data(file, FIRST_COOLDOWN_FOLDER), ignore_index=True)

In [ ]:
fig, ax2 = plt.subplots(figsize=(10, 8))

myFmt = DateFormatter("%H:%M\n%a")
ax2.xaxis.set_major_formatter(myFmt)

ax2.set_ylabel(r"Temperature (K)")
ax2.plot(dft["MPL_datetimes"], dft["Baseplate"], color="C1", label="Baseplate T")
# Sesnor location was changed
ax2.plot(dft["MPL_datetimes"], dft["Input C1"], color="C1")

ax2.plot(dft["MPL_datetimes"], dft["Magnet Base"], color="C3", label="Magnet Base T")
ax2.plot(dft["MPL_datetimes"], dft["Tip Holder"], color="C2", label="Tip Holder T")

ax2.legend()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

myFmt = DateFormatter("%H:%M\n%a")
ax.xaxis.set_major_formatter(myFmt)

# Pressure
ax.set_ylabel(r"Main Pressure (mbar)", color="C0")
ax.set_yscale("log", base=10)
ax.tick_params(axis='y', labelcolor="C0")
ax.plot(dfp["MPL_datetimes"], dfp["Main"], "-", color="C0")

x, y = sft.time_extrapolation(dfp, "Main", end_date="19-Dec-20 15:00", start_index=95000, fit="logarithmic")
ax.plot(x, y, "--")

In [ ]:
x = np.linspace(10, 1e5, 1000)
a, b = 1e2, -1
y = a + b * np.log(x)

fig, ax = plt.subplots()
ax.set_yscale("log", base=10)
ax.plot(x, y)

def func(x, a, b, c):
    return a + b * np.log(c*x)

from scipy.optimize import curve_fit
popt, pcov = curve_fit(func, xdata=x, ydata=y)

x = np.linspace(5e4, 5e5, 1000)
ax.plot(x, func(x, *popt), "--")
print(popt)

# LHe level and temperature

In [ ]:
fig, (ax, ax2) = plt.subplots(nrows=2, sharex=True, figsize=(10, 8))

dft = sio.read_tm224_data("temperature-monitoring16_LHe.xls", FIRST_COOLDOWN_FOLDER)

myFmt = DateFormatter("%H:%M")
ax.xaxis.set_major_formatter(myFmt)

ax.set_ylabel(r"Temperature (K)")
ax.plot(dft["MPL_datetimes"], dft["Baseplate"], color="C1", label="Baseplate")
ax.plot(dft["MPL_datetimes"], dft["Magnet Base"], color="C3", label="Magnet Base")
ax.plot(dft["MPL_datetimes"], dft["Tip Holder"], color="C2", label="Tip Holder")

x, y, _mod, y_mod = sft.time_extrapolation_lmfit(dft, "Baseplate", end_date="24-Dec-20 10:00", start_index=19000, fit="linear")
ax.plot(x, y, "-o", color="C1")
sft.setpointy_reach_time(x, y, 4.5)

ax.legend()

ax2.set_ylabel(r"LHe level (mm)")
time_strings = ["23-Dec-20 22:10", "23-Dec-20 22:24", "23-Dec-20 22:48", "23-Dec-20 22:58", "23-Dec-20 23:45", "24-Dec-20 00:20", "24-Dec-20 00:43"]
level = [326, 318, 310, 309, 282, 260, 238]

time_object_mpl = []
for time_string in time_strings:
    time_object_mpl.append(matplotlib.dates.date2num(datetime.datetime.strptime(time_string, "%d-%b-%y %H:%M")))

ax2.plot(time_object_mpl, level, "--o", color="C0", label="LHe level")
#sio.savefig("temperature-monitoring_level-monitoring", FIRST_COOLDOWN_FOLDER)